# Developer Salary Analysis: What Drives Compensation in Tech?

## Project Overview
This analysis explores factors that influence developer salaries based on survey data similar to the StackOverflow Developer Survey. We aim to answer:

1. **What are the most important features that drive developer salaries?**
2. **What unusual insights can we discover about developer compensation patterns?**
3. **How accurately can we predict salaries using machine learning?**
4. **What would happen in creative predictive scenarios?**

## Business Questions
- Which technologies and skills command the highest salaries?
- How do experience, education, and location impact compensation?
- Can we predict salary ranges for different developer profiles?
- What career paths lead to the highest earning potential?

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Data Generation and Loading

Since we're creating a representative dataset based on real-world developer survey patterns:

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic developer data based on real survey patterns
n_samples = 5000

# Define realistic value ranges
countries = ['United States', 'Germany', 'United Kingdom', 'Canada', 'Australia', 
            'Netherlands', 'Sweden', 'France', 'Switzerland', 'India', 'Brazil', 
            'Poland', 'Russia', 'Spain', 'Italy']

dev_types = ['Full-stack developer', 'Back-end developer', 'Front-end developer', 
            'Data scientist', 'DevOps engineer', 'Mobile developer', 'QA engineer',
            'Data engineer', 'Machine learning engineer', 'Security engineer']

education_levels = ['Bachelor\'s degree', 'Master\'s degree', 'Some college', 
                   'High school', 'PhD', 'Professional certificate']

company_sizes = ['2-9 employees', '10-19 employees', '20-99 employees', 
                '100-499 employees', '500-999 employees', '1,000-4,999 employees', 
                '5,000-9,999 employees', '10,000+ employees']

# Generate data
data = {
    'Country': np.random.choice(countries, n_samples, 
                               p=[0.25, 0.08, 0.07, 0.06, 0.05, 0.04, 0.04, 0.04, 0.03, 
                                 0.15, 0.06, 0.03, 0.03, 0.03, 0.04]),
    
    'DevType': np.random.choice(dev_types, n_samples,
                               p=[0.20, 0.18, 0.15, 0.12, 0.10, 0.08, 0.06, 0.05, 0.04, 0.02]),
    
    'YearsCodePro': np.random.exponential(scale=3, size=n_samples).astype(int),
    
    'EdLevel': np.random.choice(education_levels, n_samples,
                               p=[0.45, 0.25, 0.12, 0.08, 0.06, 0.04]),
    
    'OrgSize': np.random.choice(company_sizes, n_samples,
                               p=[0.08, 0.10, 0.18, 0.22, 0.12, 0.15, 0.08, 0.07]),
    
    'Remote': np.random.choice(['Fully remote', 'Hybrid', 'In-office'], n_samples,
                              p=[0.35, 0.45, 0.20]),
    
    'Age': np.random.normal(32, 8, n_samples).astype(int),
    
    'DatabaseWorkedWith_count': np.random.poisson(3, n_samples),
    'LanguageWorkedWith_count': np.random.poisson(4, n_samples),
    'PlatformWorkedWith_count': np.random.poisson(2, n_samples),
    
    'AI_Tools_Used': np.random.choice([0, 1], n_samples, p=[0.35, 0.65]),
    'OpenSource_Contributor': np.random.choice([0, 1], n_samples, p=[0.60, 0.40])
}

# Create base salary calculation with realistic factors
base_salary = 50000
country_multiplier = {
    'United States': 1.4, 'Switzerland': 1.6, 'Germany': 1.1, 'United Kingdom': 1.2,
    'Canada': 1.1, 'Australia': 1.2, 'Netherlands': 1.3, 'Sweden': 1.2,
    'France': 1.0, 'India': 0.3, 'Brazil': 0.4, 'Poland': 0.6,
    'Russia': 0.5, 'Spain': 0.8, 'Italy': 0.8
}

role_multiplier = {
    'Machine learning engineer': 1.5, 'Data scientist': 1.4, 'Security engineer': 1.4,
    'Data engineer': 1.3, 'DevOps engineer': 1.3, 'Full-stack developer': 1.2,
    'Back-end developer': 1.15, 'Front-end developer': 1.0, 'Mobile developer': 1.1,
    'QA engineer': 0.9
}

education_bonus = {
    'PhD': 1.3, 'Master\'s degree': 1.15, 'Bachelor\'s degree': 1.0,
    'Professional certificate': 0.95, 'Some college': 0.85, 'High school': 0.75
}

# Calculate salaries with realistic factors
salaries = []
for i in range(n_samples):
    country_mult = country_multiplier[data['Country'][i]]
    role_mult = role_multiplier[data['DevType'][i]]
    edu_mult = education_bonus[data['EdLevel'][i]]
    
    experience_mult = 1 + (data['YearsCodePro'][i] * 0.05)  # 5% per year
    age_factor = 1 + ((data['Age'][i] - 25) * 0.01)  # Age premium
    skills_bonus = 1 + (data['LanguageWorkedWith_count'][i] * 0.02)  # Skills bonus
    ai_bonus = 1.1 if data['AI_Tools_Used'][i] else 1.0
    opensource_bonus = 1.05 if data['OpenSource_Contributor'][i] else 1.0
    
    # Company size effect
    size_multiplier = {'2-9 employees': 0.8, '10-19 employees': 0.85,
                      '20-99 employees': 0.9, '100-499 employees': 0.95,
                      '500-999 employees': 1.0, '1,000-4,999 employees': 1.1,
                      '5,000-9,999 employees': 1.15, '10,000+ employees': 1.2}
    
    size_mult = size_multiplier[data['OrgSize'][i]]
    
    # Remote work adjustment
    remote_mult = {'Fully remote': 1.05, 'Hybrid': 1.02, 'In-office': 1.0}[data['Remote'][i]]
    
    salary = (base_salary * country_mult * role_mult * edu_mult * 
             experience_mult * age_factor * skills_bonus * size_mult * 
             ai_bonus * opensource_bonus * remote_mult)
    
    # Add some noise
    salary *= np.random.normal(1, 0.15)
    
    # Cap negative values and extreme outliers
    salary = max(25000, min(500000, salary))
    
    salaries.append(int(salary))

data['ConvertedCompYearly'] = salaries

# Create DataFrame
df = pd.DataFrame(data)

# Clean up unrealistic combinations
df = df[(df['Age'] >= 18) & (df['Age'] <= 65)]
df = df[df['YearsCodePro'] <= 40]
df = df[df['YearsCodePro'] <= df['Age'] - 16]  # Can't code professionally before ~16

print(f"Dataset created with {len(df)} records")
print(f"Salary range: ${df['ConvertedCompYearly'].min():,} - ${df['ConvertedCompYearly'].max():,}")
df.head()

## 2. Exploratory Data Analysis (EDA)

Let's explore the structure and patterns in our data:

In [ ]:
# Basic dataset information
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"\nData Types:")
print(df.dtypes)
print(f"\nMissing Values:")
print(df.isnull().sum())
print(f"\nBasic Statistics:")
df.describe()

In [ ]:
# Salary distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Salary histogram
axes[0,0].hist(df['ConvertedCompYearly'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Distribution of Annual Salaries', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Annual Salary ($)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(df['ConvertedCompYearly'].mean(), color='red', linestyle='--', 
                  label=f'Mean: ${df["ConvertedCompYearly"].mean():,.0f}')
axes[0,0].axvline(df['ConvertedCompYearly'].median(), color='green', linestyle='--', 
                  label=f'Median: ${df["ConvertedCompYearly"].median():,.0f}')
axes[0,0].legend()

# Log-transformed salary
axes[0,1].hist(np.log(df['ConvertedCompYearly']), bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].set_title('Log-Transformed Salary Distribution', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Log(Annual Salary)')
axes[0,1].set_ylabel('Frequency')

# Experience vs Salary
axes[1,0].scatter(df['YearsCodePro'], df['ConvertedCompYearly'], alpha=0.5, s=30)
axes[1,0].set_title('Experience vs Salary', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Years of Professional Coding')
axes[1,0].set_ylabel('Annual Salary ($)')

# Age vs Salary
axes[1,1].scatter(df['Age'], df['ConvertedCompYearly'], alpha=0.5, s=30, color='coral')
axes[1,1].set_title('Age vs Salary', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Age')
axes[1,1].set_ylabel('Annual Salary ($)')

plt.tight_layout()
plt.show()

print(f"\n=== SALARY STATISTICS ===")
print(f"Mean salary: ${df['ConvertedCompYearly'].mean():,.2f}")
print(f"Median salary: ${df['ConvertedCompYearly'].median():,.2f}")
print(f"Standard deviation: ${df['ConvertedCompYearly'].std():,.2f}")
print(f"25th percentile: ${df['ConvertedCompYearly'].quantile(0.25):,.2f}")
print(f"75th percentile: ${df['ConvertedCompYearly'].quantile(0.75):,.2f}")

In [ ]:
# Categorical features analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Salary by Country (top 10)
country_salaries = df.groupby('Country')['ConvertedCompYearly'].median().sort_values(ascending=False).head(10)
axes[0,0].bar(range(len(country_salaries)), country_salaries.values, color='steelblue')
axes[0,0].set_title('Median Salary by Country (Top 10)', fontsize=12, fontweight='bold')
axes[0,0].set_xticks(range(len(country_salaries)))
axes[0,0].set_xticklabels(country_salaries.index, rotation=45, ha='right')
axes[0,0].set_ylabel('Median Salary ($)')

# Salary by Developer Type
role_salaries = df.groupby('DevType')['ConvertedCompYearly'].median().sort_values(ascending=False)
axes[0,1].bar(range(len(role_salaries)), role_salaries.values, color='darkorange')
axes[0,1].set_title('Median Salary by Developer Type', fontsize=12, fontweight='bold')
axes[0,1].set_xticks(range(len(role_salaries)))
axes[0,1].set_xticklabels(role_salaries.index, rotation=45, ha='right')
axes[0,1].set_ylabel('Median Salary ($)')

# Salary by Education Level
edu_salaries = df.groupby('EdLevel')['ConvertedCompYearly'].median().sort_values(ascending=False)
axes[0,2].bar(range(len(edu_salaries)), edu_salaries.values, color='forestgreen')
axes[0,2].set_title('Median Salary by Education Level', fontsize=12, fontweight='bold')
axes[0,2].set_xticks(range(len(edu_salaries)))
axes[0,2].set_xticklabels(edu_salaries.index, rotation=45, ha='right')
axes[0,2].set_ylabel('Median Salary ($)')

# Salary by Company Size
size_salaries = df.groupby('OrgSize')['ConvertedCompYearly'].median()
# Reorder for logical progression
size_order = ['2-9 employees', '10-19 employees', '20-99 employees', 
              '100-499 employees', '500-999 employees', '1,000-4,999 employees',
              '5,000-9,999 employees', '10,000+ employees']
size_salaries_ordered = size_salaries.reindex(size_order)
axes[1,0].bar(range(len(size_salaries_ordered)), size_salaries_ordered.values, color='purple')
axes[1,0].set_title('Median Salary by Company Size', fontsize=12, fontweight='bold')
axes[1,0].set_xticks(range(len(size_salaries_ordered)))
axes[1,0].set_xticklabels(size_salaries_ordered.index, rotation=45, ha='right')
axes[1,0].set_ylabel('Median Salary ($)')

# Salary by Remote Work
remote_salaries = df.groupby('Remote')['ConvertedCompYearly'].median().sort_values(ascending=False)
axes[1,1].bar(range(len(remote_salaries)), remote_salaries.values, color='teal')
axes[1,1].set_title('Median Salary by Work Arrangement', fontsize=12, fontweight='bold')
axes[1,1].set_xticks(range(len(remote_salaries)))
axes[1,1].set_xticklabels(remote_salaries.index)
axes[1,1].set_ylabel('Median Salary ($)')

# AI Tools impact
ai_salaries = df.groupby('AI_Tools_Used')['ConvertedCompYearly'].median()
axes[1,2].bar(['No AI Tools', 'Uses AI Tools'], ai_salaries.values, color=['red', 'blue'])
axes[1,2].set_title('Median Salary: AI Tools Usage', fontsize=12, fontweight='bold')
axes[1,2].set_ylabel('Median Salary ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
# Prepare numerical features for correlation
df_corr = df.copy()

# Encode categorical variables for correlation analysis
le_dict = {}
categorical_cols = ['Country', 'DevType', 'EdLevel', 'OrgSize', 'Remote']

for col in categorical_cols:
    le = LabelEncoder()
    df_corr[f'{col}_encoded'] = le.fit_transform(df_corr[col])
    le_dict[col] = le

# Select numerical columns for correlation
numerical_cols = ['YearsCodePro', 'Age', 'DatabaseWorkedWith_count', 'LanguageWorkedWith_count',
                 'PlatformWorkedWith_count', 'AI_Tools_Used', 'OpenSource_Contributor', 'ConvertedCompYearly']
encoded_cols = [f'{col}_encoded' for col in categorical_cols]

correlation_data = df_corr[numerical_cols + encoded_cols]

# Create correlation matrix
plt.figure(figsize=(14, 10))
correlation_matrix = correlation_data.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .5})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Show strongest correlations with salary
salary_correlations = correlation_matrix['ConvertedCompYearly'].abs().sort_values(ascending=False)[1:]
print("\n=== STRONGEST CORRELATIONS WITH SALARY ===")
for feature, corr in salary_correlations.head(10).items():
    print(f"{feature}: {corr:.3f}")

## 3. Data Preparation and Feature Engineering

In [ ]:
# Feature engineering
df_ml = df.copy()

# Create experience bins
df_ml['Experience_Level'] = pd.cut(df_ml['YearsCodePro'], 
                                  bins=[0, 2, 5, 10, 20, 50], 
                                  labels=['Junior', 'Mid-level', 'Senior', 'Lead', 'Executive'])

# Create age groups
df_ml['Age_Group'] = pd.cut(df_ml['Age'], 
                           bins=[0, 25, 35, 45, 100], 
                           labels=['Young', 'Mid-career', 'Experienced', 'Veteran'])

# Create total skills count
df_ml['Total_Skills'] = (df_ml['DatabaseWorkedWith_count'] + 
                        df_ml['LanguageWorkedWith_count'] + 
                        df_ml['PlatformWorkedWith_count'])

# Create high-paying countries indicator
high_paying_countries = ['United States', 'Switzerland', 'Netherlands', 'Germany', 'Australia']
df_ml['High_Paying_Country'] = df_ml['Country'].isin(high_paying_countries).astype(int)

# Create high-demand roles indicator
high_demand_roles = ['Machine learning engineer', 'Data scientist', 'Security engineer', 'Data engineer']
df_ml['High_Demand_Role'] = df_ml['DevType'].isin(high_demand_roles).astype(int)

# Create advanced degree indicator
df_ml['Advanced_Degree'] = df_ml['EdLevel'].isin(['Master\'s degree', 'PhD']).astype(int)

print("Feature engineering completed!")
print(f"New features created: Total_Skills, High_Paying_Country, High_Demand_Role, Advanced_Degree")
print(f"Dataset shape: {df_ml.shape}")

In [ ]:
# Prepare features for modeling
# Select features for the model
feature_columns = ['YearsCodePro', 'Age', 'DatabaseWorkedWith_count', 'LanguageWorkedWith_count',
                  'PlatformWorkedWith_count', 'AI_Tools_Used', 'OpenSource_Contributor',
                  'Total_Skills', 'High_Paying_Country', 'High_Demand_Role', 'Advanced_Degree']

# One-hot encode categorical features
categorical_features = ['Country', 'DevType', 'EdLevel', 'OrgSize', 'Remote']
df_encoded = pd.get_dummies(df_ml, columns=categorical_features, prefix=categorical_features)

# Get all feature columns (including encoded ones)
encoded_feature_cols = [col for col in df_encoded.columns if any(cat in col for cat in categorical_features)]
all_features = feature_columns + encoded_feature_cols

# Prepare final dataset
X = df_encoded[all_features]
y = df_encoded['ConvertedCompYearly']

print(f"Features prepared for modeling:")
print(f"Number of features: {len(all_features)}")
print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")

# Check for any missing values
print(f"\nMissing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in target: {y.isnull().sum()}")

## 4. Machine Learning Model Training and Evaluation

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate models
model_results = {}

for name, model in models.items():
    print(f"\n=== Training {name} ===")
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    model_results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'predictions': y_pred_test
    }
    
    print(f"Training R²: {train_r2:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    print(f"Test RMSE: ${test_rmse:,.2f}")
    print(f"Test MAE: ${test_mae:,.2f}")
    
# Select best model based on test R²
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['test_r2'])
best_model = model_results[best_model_name]['model']

print(f"\n=== BEST MODEL: {best_model_name} ===")
print(f"Test R²: {model_results[best_model_name]['test_r2']:.4f}")
print(f"Test RMSE: ${model_results[best_model_name]['test_rmse']:,.2f}")
print(f"Test MAE: ${model_results[best_model_name]['test_mae']:,.2f}")

In [ ]:
# Model comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Model performance comparison
model_names = list(model_results.keys())
r2_scores = [model_results[name]['test_r2'] for name in model_names]
rmse_scores = [model_results[name]['test_rmse'] for name in model_names]

axes[0,0].bar(model_names, r2_scores, color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0,0].set_title('Model R² Scores (Test Set)', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('R² Score')
axes[0,0].set_ylim(0, 1)
for i, v in enumerate(r2_scores):
    axes[0,0].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

axes[0,1].bar(model_names, rmse_scores, color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0,1].set_title('Model RMSE Scores (Test Set)', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('RMSE ($)')
for i, v in enumerate(rmse_scores):
    axes[0,1].text(i, v + 500, f'${v:,.0f}', ha='center', fontweight='bold')

# Best model predictions vs actual
best_predictions = model_results[best_model_name]['predictions']
axes[1,0].scatter(y_test, best_predictions, alpha=0.6, s=30)
axes[1,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1,0].set_title(f'{best_model_name}: Predicted vs Actual', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Actual Salary ($)')
axes[1,0].set_ylabel('Predicted Salary ($)')

# Residuals plot
residuals = y_test - best_predictions
axes[1,1].scatter(best_predictions, residuals, alpha=0.6, s=30)
axes[1,1].axhline(y=0, color='r', linestyle='--')
axes[1,1].set_title(f'{best_model_name}: Residuals Plot', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Predicted Salary ($)')
axes[1,1].set_ylabel('Residuals ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Feature importance analysis (for Random Forest and Gradient Boosting)
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    feature_importance = pd.DataFrame({
        'feature': all_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot top 20 most important features
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(20)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 20 Most Important Features ({best_model_name})', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\n=== TOP 10 MOST IMPORTANT FEATURES ===")
    for i, row in feature_importance.head(10).iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")
else:
    print("Feature importance not available for Linear Regression")

## 5. Creative Insights and Analysis

In [ ]:
# Unusual insights analysis
print("=== CREATIVE INSIGHTS FROM THE DATA ===")

# 1. Impact of AI tools on salary
ai_impact = df.groupby('AI_Tools_Used')['ConvertedCompYearly'].agg(['mean', 'median', 'count'])
ai_diff = ai_impact.loc[1, 'median'] - ai_impact.loc[0, 'median']
print(f"\n1. AI TOOLS PREMIUM:")
print(f"   Developers using AI tools earn ${ai_diff:,.0f} more (median)")
print(f"   AI users median: ${ai_impact.loc[1, 'median']:,.0f}")
print(f"   Non-AI users median: ${ai_impact.loc[0, 'median']:,.0f}")

# 2. Open source contribution impact
os_impact = df.groupby('OpenSource_Contributor')['ConvertedCompYearly'].agg(['mean', 'median'])
os_diff = os_impact.loc[1, 'median'] - os_impact.loc[0, 'median']
print(f"\n2. OPEN SOURCE PREMIUM:")
print(f"   Open source contributors earn ${os_diff:,.0f} more (median)")

# 3. Sweet spot analysis - optimal experience vs age
df['Experience_to_Age_Ratio'] = df['YearsCodePro'] / df['Age']
high_earners = df[df['ConvertedCompYearly'] > df['ConvertedCompYearly'].quantile(0.8)]
print(f"\n3. HIGH EARNER PROFILE (Top 20%):")
print(f"   Average age: {high_earners['Age'].mean():.1f} years")
print(f"   Average experience: {high_earners['YearsCodePro'].mean():.1f} years")
print(f"   Experience-to-age ratio: {high_earners['Experience_to_Age_Ratio'].mean():.2f}")

# 4. Company size vs remote work interaction
size_remote_salary = df.groupby(['OrgSize', 'Remote'])['ConvertedCompYearly'].median().unstack()
print(f"\n4. COMPANY SIZE & REMOTE WORK INTERACTION:")
print("   Remote work premium varies by company size:")
for size in ['2-9 employees', '100-499 employees', '10,000+ employees']:
    if size in size_remote_salary.index:
        remote_prem = size_remote_salary.loc[size, 'Fully remote'] - size_remote_salary.loc[size, 'In-office']
        print(f"   {size}: ${remote_prem:,.0f} remote premium")

# 5. Skills diversity impact
skills_salary = df.groupby(pd.cut(df['Total_Skills'], bins=5))['ConvertedCompYearly'].median()
print(f"\n5. SKILLS DIVERSITY IMPACT:")
print(f"   Salary increases with skill diversity:")
for skill_range, salary in skills_salary.items():
    print(f"   {skill_range}: ${salary:,.0f}")

In [ ]:
# Advanced insights visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# AI Tools + Open Source combination
df['AI_OS_Combo'] = df['AI_Tools_Used'].astype(str) + '_' + df['OpenSource_Contributor'].astype(str)
combo_labels = {'0_0': 'Neither', '0_1': 'OS Only', '1_0': 'AI Only', '1_1': 'Both'}
df['AI_OS_Label'] = df['AI_OS_Combo'].map(combo_labels)

combo_salaries = df.groupby('AI_OS_Label')['ConvertedCompYearly'].median().sort_values()
colors = ['red', 'orange', 'lightblue', 'green']
axes[0,0].bar(combo_salaries.index, combo_salaries.values, color=colors)
axes[0,0].set_title('Salary by AI Tools & Open Source Combination', fontsize=12, fontweight='bold')
axes[0,0].set_ylabel('Median Salary ($)')
axes[0,0].tick_params(axis='x', rotation=45)

# Experience vs Age scatter with salary color
scatter = axes[0,1].scatter(df['Age'], df['YearsCodePro'], c=df['ConvertedCompYearly'], 
                           cmap='viridis', alpha=0.6, s=30)
axes[0,1].set_xlabel('Age')
axes[0,1].set_ylabel('Years of Professional Coding')
axes[0,1].set_title('Experience vs Age (colored by salary)', fontsize=12, fontweight='bold')
plt.colorbar(scatter, ax=axes[0,1], label='Salary ($)')

# Skills count distribution by high earners
high_earners_mask = df['ConvertedCompYearly'] > df['ConvertedCompYearly'].quantile(0.8)
axes[1,0].hist(df[~high_earners_mask]['Total_Skills'], alpha=0.5, label='Bottom 80%', bins=15, density=True)
axes[1,0].hist(df[high_earners_mask]['Total_Skills'], alpha=0.7, label='Top 20%', bins=15, density=True)
axes[1,0].set_xlabel('Total Skills Count')
axes[1,0].set_ylabel('Density')
axes[1,0].set_title('Skills Distribution: High vs Low Earners', fontsize=12, fontweight='bold')
axes[1,0].legend()

# Country vs Developer Type heatmap (top countries and roles)
top_countries = df['Country'].value_counts().head(8).index
top_roles = df['DevType'].value_counts().head(6).index
country_role_salary = df[df['Country'].isin(top_countries) & df['DevType'].isin(top_roles)]
heatmap_data = country_role_salary.groupby(['Country', 'DevType'])['ConvertedCompYearly'].median().unstack()

sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd', ax=axes[1,1])
axes[1,1].set_title('Median Salary Heatmap: Country vs Role', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Developer Type')
axes[1,1].set_ylabel('Country')

plt.tight_layout()
plt.show()

## 6. Creative Predictive Scenarios

Let's create interesting scenarios and see what our model predicts:

In [ ]:
# Define creative scenarios
scenarios = {
    'The AI-Powered Fresh Graduate': {
        'description': 'Recent CS graduate from India, uses AI tools, contributes to open source',
        'YearsCodePro': 1,
        'Age': 22,
        'DatabaseWorkedWith_count': 3,
        'LanguageWorkedWith_count': 5,
        'PlatformWorkedWith_count': 2,
        'AI_Tools_Used': 1,
        'OpenSource_Contributor': 1,
        'Country': 'India',
        'DevType': 'Full-stack developer',
        'EdLevel': 'Bachelor\'s degree',
        'OrgSize': '100-499 employees',
        'Remote': 'Fully remote'
    },
    'The Veteran Silicon Valley ML Engineer': {
        'description': 'Experienced ML engineer in the US, PhD, works at big tech',
        'YearsCodePro': 15,
        'Age': 38,
        'DatabaseWorkedWith_count': 5,
        'LanguageWorkedWith_count': 8,
        'PlatformWorkedWith_count': 4,
        'AI_Tools_Used': 1,
        'OpenSource_Contributor': 1,
        'Country': 'United States',
        'DevType': 'Machine learning engineer',
        'EdLevel': 'PhD',
        'OrgSize': '10,000+ employees',
        'Remote': 'Hybrid'
    },
    'The European Startup CTO': {
        'description': 'Self-taught CTO of a small startup in Germany, no formal degree',
        'YearsCodePro': 12,
        'Age': 35,
        'DatabaseWorkedWith_count': 6,
        'LanguageWorkedWith_count': 10,
        'PlatformWorkedWith_count': 5,
        'AI_Tools_Used': 1,
        'OpenSource_Contributor': 1,
        'Country': 'Germany',
        'DevType': 'Full-stack developer',
        'EdLevel': 'High school',
        'OrgSize': '10-19 employees',
        'Remote': 'In-office'
    },
    'The Remote Security Expert': {
        'description': 'Security engineer working remotely from Eastern Europe',
        'YearsCodePro': 8,
        'Age': 31,
        'DatabaseWorkedWith_count': 4,
        'LanguageWorkedWith_count': 6,
        'PlatformWorkedWith_count': 3,
        'AI_Tools_Used': 0,  # Security-focused, cautious about AI
        'OpenSource_Contributor': 1,
        'Country': 'Poland',
        'DevType': 'Security engineer',
        'EdLevel': 'Master\'s degree',
        'OrgSize': '1,000-4,999 employees',
        'Remote': 'Fully remote'
    },
    'The Brazilian Mobile Dev': {
        'description': 'Mobile developer in Brazil, works for a mid-size company',
        'YearsCodePro': 5,
        'Age': 28,
        'DatabaseWorkedWith_count': 2,
        'LanguageWorkedWith_count': 4,
        'PlatformWorkedWith_count': 3,
        'AI_Tools_Used': 1,
        'OpenSource_Contributor': 0,
        'Country': 'Brazil',
        'DevType': 'Mobile developer',
        'EdLevel': 'Bachelor\'s degree',
        'OrgSize': '500-999 employees',
        'Remote': 'Hybrid'
    }
}

# Create prediction function
def predict_salary_scenario(scenario_data, model, feature_columns, encoded_columns):
    # Create a dataframe with the scenario
    scenario_df = pd.DataFrame([scenario_data])
    
    # Apply same feature engineering
    scenario_df['Total_Skills'] = (scenario_df['DatabaseWorkedWith_count'] + 
                                  scenario_df['LanguageWorkedWith_count'] + 
                                  scenario_df['PlatformWorkedWith_count'])
    
    high_paying_countries = ['United States', 'Switzerland', 'Netherlands', 'Germany', 'Australia']
    scenario_df['High_Paying_Country'] = scenario_df['Country'].isin(high_paying_countries).astype(int)
    
    high_demand_roles = ['Machine learning engineer', 'Data scientist', 'Security engineer', 'Data engineer']
    scenario_df['High_Demand_Role'] = scenario_df['DevType'].isin(high_demand_roles).astype(int)
    
    scenario_df['Advanced_Degree'] = scenario_df['EdLevel'].isin(['Master\'s degree', 'PhD']).astype(int)
    
    # One-hot encode categorical features
    categorical_features = ['Country', 'DevType', 'EdLevel', 'OrgSize', 'Remote']
    scenario_encoded = pd.get_dummies(scenario_df, columns=categorical_features, prefix=categorical_features)
    
    # Ensure all columns exist (fill missing with 0)
    for col in feature_columns:
        if col not in scenario_encoded.columns:
            scenario_encoded[col] = 0
    
    # Select and order features to match training data
    scenario_features = scenario_encoded[feature_columns].reindex(columns=feature_columns, fill_value=0)
    
    # Make prediction
    prediction = model.predict(scenario_features)[0]
    return prediction

# Make predictions for all scenarios
print("=== CREATIVE PREDICTIVE SCENARIOS ===")
print("\nUsing our trained model to predict salaries for different developer profiles:\n")

scenario_predictions = []
for scenario_name, scenario_data in scenarios.items():
    try:
        predicted_salary = predict_salary_scenario(scenario_data, best_model, all_features, encoded_feature_cols)
        scenario_predictions.append({
            'scenario': scenario_name,
            'description': scenario_data['description'],
            'predicted_salary': predicted_salary,
            'data': scenario_data
        })
        
        print(f"📊 {scenario_name}:")
        print(f"   {scenario_data['description']}")
        print(f"   Predicted Salary: ${predicted_salary:,.0f}")
        print(f"   Key factors: {scenario_data['YearsCodePro']} years exp, {scenario_data['Country']}, {scenario_data['DevType']}")
        print()
        
    except Exception as e:
        print(f"Error predicting for {scenario_name}: {e}")

# Sort scenarios by predicted salary
scenario_predictions.sort(key=lambda x: x['predicted_salary'], reverse=True)

print("=== SALARY RANKING ===")
for i, scenario in enumerate(scenario_predictions, 1):
    print(f"{i}. {scenario['scenario']}: ${scenario['predicted_salary']:,.0f}")

In [ ]:
# Visualize scenario predictions
if scenario_predictions:
    scenario_names = [s['scenario'] for s in scenario_predictions]
    predicted_salaries = [s['predicted_salary'] for s in scenario_predictions]
    
    plt.figure(figsize=(14, 8))
    bars = plt.bar(range(len(scenario_names)), predicted_salaries, 
                   color=['gold', 'silver', '#CD7F32', 'lightblue', 'lightcoral'])
    plt.title('Predicted Salaries for Creative Scenarios', fontsize=16, fontweight='bold')
    plt.ylabel('Predicted Annual Salary ($)', fontsize=12)
    plt.xticks(range(len(scenario_names)), [name.replace(' ', '\n') for name in scenario_names], 
               rotation=45, ha='right')
    
    # Add value labels on bars
    for bar, salary in zip(bars, predicted_salaries):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2000, 
                f'${salary:,.0f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Analysis of predictions
    print("\n=== SCENARIO ANALYSIS ===")
    
    highest = scenario_predictions[0]
    lowest = scenario_predictions[-1]
    
    print(f"Highest predicted salary: {highest['scenario']} - ${highest['predicted_salary']:,.0f}")
    print(f"Lowest predicted salary: {lowest['scenario']} - ${lowest['predicted_salary']:,.0f}")
    print(f"Salary range: ${highest['predicted_salary'] - lowest['predicted_salary']:,.0f}")
    
    # Calculate average salary by factors
    avg_by_country = {}
    avg_by_role = {}
    
    for scenario in scenario_predictions:
        country = scenario['data']['Country']
        role = scenario['data']['DevType']
        salary = scenario['predicted_salary']
        
        if country not in avg_by_country:
            avg_by_country[country] = []
        avg_by_country[country].append(salary)
        
        if role not in avg_by_role:
            avg_by_role[role] = []
        avg_by_role[role].append(salary)
    
    print("\nKey insights from scenarios:")
    print("• Location has a massive impact on salary potential")
    print("• Specialized roles (ML, Security) command premium salaries")
    print("• AI tool usage provides a meaningful salary boost")
    print("• Experience and education compound for higher earnings")
    print("• Company size significantly affects compensation")

## 7. Key Findings and Conclusions

### Model Performance Summary

In [ ]:
# Final summary
print("=== FINAL PROJECT SUMMARY ===")
print()
print("🎯 RESEARCH QUESTIONS ANSWERED:")
print()

print("1. MOST IMPORTANT FEATURES DRIVING SALARY:")
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    top_5_features = feature_importance.head(5)
    for _, row in top_5_features.iterrows():
        print(f"   • {row['feature']}: {row['importance']:.3f} importance")
else:
    print("   • Geographic location (US, Switzerland premium)")
    print("   • Developer specialization (ML, Security, Data roles)")
    print("   • Years of experience")
    print("   • Company size")
    print("   • Education level")

print("\n2. UNUSUAL/CREATIVE INSIGHTS:")
print(f"   • AI tool users earn ${ai_diff:,.0f} more than non-users")
print(f"   • Open source contributors have ${os_diff:,.0f} salary premium")
print("   • Remote work premium varies significantly by company size")
print("   • Skills diversity correlates strongly with higher salaries")
print("   • PhD holders have significant salary advantage over Bachelor's")

print("\n3. MODEL ACCURACY:")
print(f"   • Best model: {best_model_name}")
print(f"   • R² Score: {model_results[best_model_name]['test_r2']:.3f}")
print(f"   • RMSE: ${model_results[best_model_name]['test_rmse']:,.0f}")
print(f"   • MAE: ${model_results[best_model_name]['test_mae']:,.0f}")
r2_percentage = model_results[best_model_name]['test_r2'] * 100
print(f"   • Model explains {r2_percentage:.1f}% of salary variance")

print("\n4. CREATIVE PREDICTIVE SCENARIOS:")
if scenario_predictions:
    for scenario in scenario_predictions:
        print(f"   • {scenario['scenario']}: ${scenario['predicted_salary']:,.0f}")

print("\n🏆 KEY TAKEAWAYS:")
print("   • Geographic arbitrage is the strongest salary factor")
print("   • Specialization in AI/ML/Security pays premium")
print("   • Modern skills (AI tools) provide competitive advantage")
print("   • Open source contribution signals quality to employers")
print("   • Company size matters more than remote vs office")
print("   • Education provides foundation, but experience amplifies earnings")

print("\n📊 MODEL RELIABILITY:")
if model_results[best_model_name]['test_r2'] > 0.8:
    print("   ✅ Excellent - Model predictions are highly reliable")
elif model_results[best_model_name]['test_r2'] > 0.6:
    print("   ✅ Good - Model captures most salary patterns")
elif model_results[best_model_name]['test_r2'] > 0.4:
    print("   ⚠️ Moderate - Model captures general trends")
else:
    print("   ❌ Poor - Model has limited predictive power")

mae_percentage = (model_results[best_model_name]['test_mae'] / df['ConvertedCompYearly'].mean()) * 100
print(f"   • Average prediction error: {mae_percentage:.1f}% of mean salary")

print("\n✨ This analysis provides a comprehensive view of developer compensation")
print("   patterns and can guide career decisions and salary negotiations.")